<a href="https://colab.research.google.com/github/Manekineco1/GLANDA/blob/main/Product_ORMC%2C_pytesseract%2C_cv2_%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Разработка решения по распознаванию донорских справок для Donorsearch, команда 10, "Это для справки"

# Написание функций распознавания и загрузки документов

Импортируем все необходимые библиотеки

In [ ]:
%%capture
!sudo apt install tesseract-ocr
!sudo apt-get install tesseract-ocr-rus
!pip install pytesseract

In [ ]:
import os, fnmatch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytesseract
import cv2
import scipy
from scipy.signal import find_peaks
import re
from google.colab import files

Расшифровываем аббревиатуры.

In [ ]:
def vddon(vid):
  if len(vid) == 0:
    return(vid)
  if vid[0].lower() == 'к':
    return 'Цельная кровь'
  if vid[0].lower() == 'п':
    return 'Плазма'
  if vid[0].lower() == 'ц':
    return 'Тромбоциты'
    return('Цельная кровь')

Прописываем функцию для распознавания документов компьютерным зрением.

In [ ]:
class ORMC():
    """
    Оптическое распознование медицинских справок
    """
    def __init__(self, namefileimg):
        self.namefileimg = namefileimg
        self.img = cv2.imread(self.namefileimg)

    def scale(self, size = 3000):
      if self.img.shape[0] < size:
        scale_percent = 3000 // self.img.shape[0]
        width = int(self.img.shape[1] * scale_percent)
        height = int(self.img.shape[0] * scale_percent)
        img = cv2.resize(self.img, (width, height),
                         interpolation = cv2.INTER_AREA
                         )
        return self.img

    def predict_raw(self):
      self.text = pytesseract.image_to_string(self.img,
                                              lang='rus',
                                              config='--psm 6'
                                              )
      return self.text



    def predict_data(self):
        ptrn1 = r'([0-9]{2}[.| ][0-9]{2}[.| ][0-9]{4})|([0-9]{8})'
        ptrn2 = '[^0-9-а-яА-Я]+'
        mtch = re.findall(ptrn1, self.text)
        if mtch:
          start = self.text.find( ''.join(mtch[0]))
          end = self.text.rfind( ''.join(mtch[-1]))
          donardates =re.sub(pattern=ptrn2,
                                  repl=' ',
                                  string= self.text[start:end]
                                  )
          rzlt = []
          str1 =  donardates.split(' ')

          for i in range(len(str1)):
            dt = []
            if str1[i].isnumeric() and re.match(r'(20[0-2]\d)',str1[i]):
              dt = [f'{str1[i-2]}.{str1[i-1]}.{str1[i]}',
                           vddon(str1[i+1])]

            if str1[i].isnumeric() and len(str1[i])> 7:
              dt = [f'{str1[i][-8:-6]}.{str1[i][-6:-4]}.{str1[i][-4:]}',
                           vddon(str1[i+1])]

            if len(dt) ==  0 and len(str1[i]) > 8 and ('-' in  str1[i]):
              dt = [f'{str1[i][-9:-7]}.{str1[i][-7:-5]}.{str1[i][-4:]}',vddon(str1[i+1])]

            if len(dt) > 0:
              if len(dt[0]) > 10:
                dt[0] = f'{dt[0][-9:-7]}.{dt[0][-7:-5]}.{dt[0][-4:]}'
              rzlt.append(dt)

          self.donardates = pd.DataFrame(rzlt, columns = ['Дата донации', 'Класс крови'])
          self.donardates['Тип донации'] = 'Безвозмездно'
          return  self.donardates
        return None


    def area(self, part = None):
      img = self.img
      if part == 'bottom':
        #нижняя часть дока
        img = img[img.shape[0]//2:img.shape[0], 0:img.shape[1]]
      if part == 'top':
        #верхняя часть дока
        img = img[img.shape[0]//2:img.shape[0], 0:img.shape[1]]
      self.img = img
      return self.img

    def drop_lines(self):
      """удаляет линии таблицы"""
      gray = cv2.cvtColor(self.img, cv2.COLOR_BGR2GRAY)
      thresh = cv2.threshold(gray,
                             0, 255,
                             cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
      # Repair horizontal table lines
      kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,1))
      thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=1)
      # Remove horizontal lines
      horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (55,2))
      detect_horizontal = cv2.morphologyEx(thresh,
                                           cv2.MORPH_OPEN,
                                           horizontal_kernel,
                                           iterations=5
                                           )
      cnts = cv2.findContours(detect_horizontal,
                              cv2.RETR_EXTERNAL,
                              cv2.CHAIN_APPROX_SIMPLE
                              )
      cnts = cnts[0] if len(cnts) == 2 else cnts[1]
      for c in cnts:
          cv2.drawContours(self.img, [c], -1, (255,255,255), 9)
      # Remove vertical lines
      vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,55))
      detect_vertical = cv2.morphologyEx(thresh,
                                         cv2.MORPH_OPEN,
                                         vertical_kernel,
                                         iterations=2)
      cnts = cv2.findContours(detect_vertical,
                              cv2.RETR_EXTERNAL,
                              cv2.CHAIN_APPROX_SIMPLE)
      cnts = cnts[0] if len(cnts) == 2 else cnts[1]
      for c in cnts:
          cv2.drawContours(self.img, [c], -1, (255,255,255), 9)
      return self.img

    def show(self, figsize=(10,14)):
      """показывает изображение сканированого документа """
      plt.figure(figsize=figsize)
      plt.imshow(self.img)
      plt.axis('off')
      plt.show()

    def fit(self):
      self.area('bottom'),
      self.drop_lines(),
      self.predict_raw(),
      df = self.predict_data()
      return df

Прописываем функцию загрузки файлов

In [ ]:
def load():
  uploaded = files.upload()
  filename = next(iter(uploaded))
  return filename

Выводим результат загрузки

In [ ]:
if __name__ == "__main__":
  filename = load()
  img = ORMC(filename)
  rezult = img.fit()
  print(rezult)

Saving 238716.jpg to 238716 (2).jpg
   Дата донации    Класс крови   Тип донации
0    29.04.2008  Цельная кровь  Безвозмездно
1    10.08.2017  Цельная кровь  Безвозмездно
2    20.10.2020  Цельная кровь  Безвозмездно
3    05.10.2013  Цельная кровь  Безвозмездно
4    16.12.2017  Цельная кровь  Безвозмездно
5    23.12.2020  Цельная кровь  Безвозмездно
6    27.03.2014  Цельная кровь  Безвозмездно
7    26.04.2018  Цельная кровь  Безвозмездно
8    17.02.2021         Плазма  Безвозмездно
9    29.07.2015  Цельная кровь  Безвозмездно
10   12.07.2018  Цельная кровь  Безвозмездно
11   03.03.2021  Цельная кровь  Безвозмездно
12   03.10.2015  Цельная кровь  Безвозмездно
13   15.11.2018  Цельная кровь  Безвозмездно
14   04.05.2021  Цельная кровь  Безвозмездно
15   18.02.2016  Цельная кровь  Безвозмездно
16   27.03.2019  Цельная кровь  Безвозмездно
17   04.08.2021  Цельная кровь  Безвозмездно
18   19.04.2016  Цельная кровь  Безвозмездно
19   06.06.2019  Цельная кровь  Безвозмездно
20   29.10.2021  Це

Промежуточный вывод:
- Нам в работу переданы 16 файлов в формате jpg, в которых нам нужно распознать данные в трех колонках - 'Дата донации', 'Класс крови',  'Тип донации'.
- Нами была написана функция по распознаванию донорских справок в формате jpg с помощью библиотеки pytesseract
- Мы загрузили необходимые файлы через гугл-диск



##Проверка

Подключаем гугл-диск

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Указываем путь к файлам

In [ ]:
PATH = '/content/drive/MyDrive/Colab Notebooks/datasets/405/405'
content = os.listdir(PATH)

Указываем, какие файлы подтягивать из директоии

In [ ]:
pattern1 = r'([0-9]{2}[.| ][0-9]{2}[.| ][0-9]{4})|([0-9]{8})'
imspv = []
pattern = "*.jpg"
for entry in content:
    if fnmatch.fnmatch(entry, pattern):
            imspv.append(entry)
imspv

['256838.jpg',
 '240493.jpg',
 '245365 .jpg',
 '204119 .jpg',
 '228963 .jpg',
 '254586 .jpg',
 '231820 .jpg',
 '236000 .jpg',
 '227414.jpg',
 '213950.jpg',
 '256578 .jpg',
 '225629 .jpg',
 '141899 .jpg',
 '243478 .jpg',
 '238716.jpg',
 '233749 .jpg']

Применяем метод распознавания к списку файлов и создаем новый экземпляр класса.

In [ ]:
rzltdf = {}
for f in imspv:
  img = ORMC(f'{PATH}/{f}')
  rzltdf[f[:6]] = img.fit()

Выводим результат распознавания

In [ ]:
for img in rzltdf:
  print()
  print("\033[1m" +img+ "\033[0m")
  print(rzltdf[img])
  print('#'*50)


256838
  Дата донации    Класс крови   Тип донации
0   27.04.2022  Цельная кровь  Безвозмездно
1   25.07.2022  Цельная кровь  Безвозмездно
2   03.09.2020  Цельная кровь  Безвозмездно
3   29.09.2022           None  Безвозмездно
4   08.07.2021  Цельная кровь  Безвозмездно
##################################################

240493
None
##################################################

245365
   Дата донации    Класс крови   Тип донации
0    14.01.2009  Цельная кровь  Безвозмездно
1    16.06.2017  Цельная кровь  Безвозмездно
2    20.04.2019  Цельная кровь  Безвозмездно
3    14.07.2009  Цельная кровь  Безвозмездно
4    28.08.2017  Цельная кровь  Безвозмездно
5    17.08.2019  Цельная кровь  Безвозмездно
6    25.01.2010  Цельная кровь  Безвозмездно
7    03.11.2017  Цельная кровь  Безвозмездно
8    22.03.2021  Цельная кровь  Безвозмездно
9    07.02.2011  Цельная кровь  Безвозмездно
10   27.01.2018  Цельная кровь  Безвозмездно
11   01.06.2021  Цельная кровь  Безвозмездно
12   08.08.2011  Цел

Подтягиваем CSV файлы с разметкой из директории

In [ ]:
csvspv = []
import os, fnmatch
content = os.listdir(PATH)
pattern = "*.csv"
for entry in content:
    if fnmatch.fnmatch(entry, pattern):
            csvspv.append(entry)
csvspv

['231820.csv',
 '227414.csv',
 '245365.csv',
 '256578.csv',
 '225629.csv',
 '236000.csv',
 '213950.csv',
 '238716.csv',
 '204119.csv',
 '228963.csv',
 '233749.csv',
 '256838.csv',
 '141899.csv',
 '254586.csv',
 '243478.csv']

Читаем CSV файлы

In [ ]:
csvdf = {}
for csv in csvspv:
  csvdf[csv[:6]]=pd.read_csv(f'{PATH}/{csv}')

Пишем функцию метрики accuracy, которая сравнивает содержимое ячеек в трех столбцах - 'Дата донации', 'Класс крови', 'Тип донации' в распознанных файлах и в файлах с разметкой . Выводим метрику для каждого файла и общую.

In [ ]:
def mtrc():
  mtrc = []
  number_coinciden = 0
  total_elements = 0
  for csv in csvdf:
    print()
    print("\033[1m" +csv+ "\033[0m")
    if csv in rzltdf.keys() and rzltdf[csv] is not None:
      test = set(csvdf[csv]['Дата донации'] +' '+ csvdf[csv]['Класс крови'] + ' ' + csvdf[csv]['Тип донации'] )
      predict = set(rzltdf[csv]['Дата донации'] +' '+ rzltdf[csv]['Класс крови'] + ' ' + rzltdf[csv]['Тип донации'] )
      total_elements +=len(test)
      number_coinciden += len(test& predict)
      mtrk = len(test& predict) / len(test)
      mtrc.append(mtrk)
      print(round(mtrk,4))

  print()
  print('Общая Метрика')
  return round(number_coinciden/total_elements,2)
mtrc()


231820
0.8462

227414

245365
0.9189

256578

225629
0.8333

236000
0.4

213950
0.7273

238716
0.9722

204119
0.6667

228963

233749
0.4286

256838
0.1818

141899
0.0

254586
0.0909

243478

Общая Метрика


0.6

Общий вывод:
- Нами была проведена работа по распознаванию донорских справок в формате jpg  с помощью библиотеки pytesseract
- Написана функция метрики accuracy, которая сравнивает содержимое ячеек в трех столбцах - 'Дата донации', 'Класс крови', 'Тип донации'.
- Выведена метрика для каждого файла и общая.
- Общая accuracy составила 0.6.
- В силу того, что в нашей команде изначально было три человека, но один из участников не работал над проектом, и потом ему искали замену и только 5 июля к нам присоединился третий участник - мы не успели отладить методы предобработки изображений, поэтому убрали соотвествующий код. А так же мы не успели обернуть проект в микросервис. Просим дать нам время до 21.07.23. За это время мы сможем улучшить метрику и скорее всего успеем обернуть в микросервис.